In [1]:
import pandas as pd
import numpy as np

## 1D gene expression profiles loading (X)



In [2]:
brca_ex_pws = pd.read_hdf('ex_pws.h5', key='brca')

In [3]:
non_brca_ex_pws = pd.read_hdf('ex_pws.h5', key='non_brca')

In [4]:
print('There are {} brca and {} non-brca gene-expression-profiles'.format(len(brca_ex_pws), len(non_brca_ex_pws)))

There are 1212 brca and 9224 non-brca gene-expression-profiles


In [5]:
pws = list()
genes = list()
for col in brca_ex_pws.columns:
    gene_pw = col.split('_')
    genes.append(gene_pw[0])
    pws.append(gene_pw[1])
    
pws = list(set(pws))
genes = list(set(genes))

In [6]:
len(pws)

33

In [7]:
len(genes)

1492

## Visualizing some 2D vectors

## Survival data loading (Y)

In [8]:
brca_clinical = pd.read_hdf('../data/TCGA_data.h5', key='brca_clinical')
non_brca_clinical = pd.read_hdf('../data/TCGA_data.h5', key='non_brca_clinical')

In [9]:
len(brca_clinical)+len(non_brca_clinical)

10497

In [10]:
# Deleting samples with 'NaN's in OS-columns 

## non_brca
non_brca_clinical['OS.time'] = non_brca_clinical['OS.time'].map(lambda x: np.nan if x == 'NaN' else x)
non_brca_clinical['OS'] = non_brca_clinical['OS'].map(lambda x: np.nan if x == 'NaN' else x)
non_brca_clinical.dropna(subset=['OS.time', 'OS'], inplace=True)
non_brca_clinical['OS.time'] = non_brca_clinical['OS.time'].astype(float)
non_brca_clinical['OS'] = non_brca_clinical['OS'].astype(float)

## brca
brca_clinical['OS.time'] = brca_clinical['OS.time'].map(lambda x: np.nan if x == 'NaN' else x)
brca_clinical['OS'] = brca_clinical['OS'].map(lambda x: np.nan if x == 'NaN' else x)
brca_clinical.dropna(subset=['OS.time', 'OS'], inplace=True)
brca_clinical['OS.time'] = brca_clinical['OS.time'].astype(float)
brca_clinical['OS'] = brca_clinical['OS'].astype(float)

In [11]:
print('There are {} brca clinical info and {} non-brca clinical info'.format(len(brca_clinical), len(non_brca_clinical)))

There are 1196 brca clinical info and 9224 non-brca clinical info


## Creating X and Y numpy-arrays

Note that the number of observations in clinical datasets is not the same as in gene-expression datasets

In [12]:
common_brca = set(brca_ex_pws.index).intersection(brca_clinical.index)

In [13]:
len(common_brca)

1196

In [14]:
brca_ex_pws = brca_ex_pws.loc[common_brca]

In [15]:
brca_ex_pws = brca_ex_pws.sort_index()

In [16]:
brca_clinical = brca_clinical.sort_index()

In [17]:
common_non_brca = set(non_brca_ex_pws.index).intersection(non_brca_clinical.index)

In [18]:
len(common_non_brca)

9224

In [19]:
non_brca_ex_pws = non_brca_ex_pws.loc[common_non_brca]

In [20]:
non_brca_ex_pws = non_brca_ex_pws.sort_index()

In [21]:
non_brca_clinical = non_brca_clinical.sort_index()

In [22]:
non_brca_ex_pws.isnull().any().any()

False

#### From OS to Survival

In [23]:
# Create 39 equally spaced intervals for follow-up time, from 0 to 5 years:
breaks = np.arange(0.,365.*5,365./8)
n_intervals = len(breaks)-1
timegap = breaks[1:] - breaks[:-1]

In [24]:
import nnet_survival
non_brca_y = nnet_survival.make_surv_array(non_brca_clinical['OS.time'],
                             non_brca_clinical['OS'],
                             breaks)
brca_y = nnet_survival.make_surv_array(brca_clinical['OS.time'],
                             brca_clinical['OS'],
                             breaks)

Using TensorFlow backend.


### Spliting data for training and testing

##### NON-BRCA

In [25]:
# NON-BRCA
# Split dataset intor train and test
from sklearn.model_selection import train_test_split

X_non_brca_train, X_non_brca_test, y_non_brca_train, y_non_brca_test, train_index, test_index  = train_test_split(non_brca_ex_pws, non_brca_y, np.arange(len(non_brca_ex_pws)), test_size=0.2, random_state=42)
X_non_brca_train, X_non_brca_val, y_non_brca_train, y_non_brca_val, train_index, val_index = train_test_split(X_non_brca_train, y_non_brca_train, train_index, test_size=0.2, random_state=44)

In [26]:
y_non_brca_train = np.array(y_non_brca_train, dtype=int)
y_non_brca_test = np.array(y_non_brca_test, dtype=int)
y_non_brca_val = np.array(y_non_brca_val, dtype=int)
y_non_brca = np.array(non_brca_y, dtype=int)

In [27]:
print("Dataset size:", X_non_brca_train.shape, X_non_brca_test.shape, X_non_brca_val.shape)

Dataset size: (5903, 49236) (1845, 49236) (1476, 49236)


In [28]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()

X_non_brca_train = sc.fit_transform(X_non_brca_train)
X_non_brca_test = sc.transform(X_non_brca_test)
X_non_brca_val = sc.transform(X_non_brca_val)
X_non_brca = sc.transform(non_brca_ex_pws)

In [29]:
del non_brca_ex_pws # to save memory

Get the train/test/val datasets in (n_samples, n_features, n_pathways) shape:

In [30]:
X_non_brca_train_2D = np.zeros([X_non_brca_train.shape[0], len(genes), len(pws)])
X_non_brca_test_2D = np.zeros([X_non_brca_test.shape[0], len(genes), len(pws)])
X_non_brca_val_2D = np.zeros([X_non_brca_val.shape[0], len(genes), len(pws)])
X_non_brca_2D = np.zeros([X_non_brca.shape[0], len(genes), len(pws)])

for i in range(X_non_brca_train.shape[0]): # samples
    for j in range(len(pws)): # pathways
        X_non_brca_train_2D[i, :, j] = X_non_brca_train[i, j*len(genes):(j+1)*len(genes)]

for i in range(X_non_brca_test.shape[0]): # samples
    for j in range(len(pws)): # pathways
        X_non_brca_test_2D[i, :, j] = X_non_brca_test[i, j*len(genes):(j+1)*len(genes)]

for i in range(X_non_brca_val.shape[0]): # samples
    for j in range(len(pws)): # pathways
        X_non_brca_val_2D[i, :, j] = X_non_brca_val[i, j*len(genes):(j+1)*len(genes)]

for i in range(X_non_brca.shape[0]): # samples
    for j in range(len(pws)): # pathways
        X_non_brca_2D[i, :, j] = X_non_brca[i, j*len(genes):(j+1)*len(genes)]

In [31]:
X_non_brca_train, X_non_brca_test, X_non_brca_val = X_non_brca_train_2D, X_non_brca_test_2D, X_non_brca_val_2D

In [32]:
X_non_brca = X_non_brca_2D

In [33]:
print("Dataset size:", X_non_brca_train.shape, X_non_brca_test.shape, X_non_brca_val.shape)

Dataset size: (5903, 1492, 33) (1845, 1492, 33) (1476, 1492, 33)


In [34]:
X_non_brca.shape

(9224, 1492, 33)

##### BRCA

In [35]:
# BRCA
# Split dataset intor train and test
from sklearn.model_selection import train_test_split

X_brca_train, X_brca_test, y_brca_train, y_brca_test, train_index, test_index  = train_test_split(brca_ex_pws, brca_y, np.arange(len(brca_ex_pws)), test_size=0.2, random_state=42)
X_brca_train, X_brca_val, y_brca_train, y_brca_val, train_index, val_index = train_test_split(X_brca_train, y_brca_train, train_index, test_size=0.2, random_state=44)

In [36]:
y_brca_train = np.array(y_brca_train, dtype=int)
y_brca_test = np.array(y_brca_test, dtype=int)
y_brca_val = np.array(y_brca_val, dtype=int)

In [37]:
print("Dataset size:", X_brca_train.shape, X_brca_test.shape, X_brca_val.shape)

Dataset size: (764, 49236) (240, 49236) (192, 49236)


In [38]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()

X_brca_train = sc.fit_transform(X_brca_train)
X_brca_test = sc.transform(X_brca_test)
X_brca_val = sc.transform(X_brca_val)

In [39]:
del brca_ex_pws # to save memory

Get the train/test/val datasets in (n_samples, n_features, n_pathways) shape:

In [40]:
X_brca_train_2D = np.zeros([X_brca_train.shape[0], len(genes), len(pws)])
X_brca_test_2D = np.zeros([X_brca_test.shape[0], len(genes), len(pws)])
X_brca_val_2D = np.zeros([X_brca_val.shape[0], len(genes), len(pws)])

for i in range(X_brca_train.shape[0]): # samples
    for j in range(len(pws)): # pathways
        X_brca_train_2D[i, :, j] = X_brca_train[i, j*len(genes):(j+1)*len(genes)]

for i in range(X_brca_test.shape[0]): # samples
    for j in range(len(pws)): # pathways
        X_brca_test_2D[i, :, j] = X_brca_test[i, j*len(genes):(j+1)*len(genes)]

for i in range(X_brca_val.shape[0]): # samples
    for j in range(len(pws)): # pathways
        X_brca_val_2D[i, :, j] = X_brca_val[i, j*len(genes):(j+1)*len(genes)]

In [41]:
X_brca_train, X_brca_test, X_brca_val = X_brca_train_2D, X_brca_test_2D, X_brca_val_2D

In [42]:
print("Dataset size:", X_brca_train.shape, X_brca_test.shape, X_brca_val.shape)

Dataset size: (764, 1492, 33) (240, 1492, 33) (192, 1492, 33)


# TRAINING MODELS

In [43]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Input, Dense, Conv1D, MaxPool1D, Activation, Dropout, Flatten
from keras import regularizers, optimizers
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
import warnings
from keras.regularizers import l1
from keras.optimizers import Adam
import tensorflow as tf

In [44]:
# Using C-index as evaluation metric (using a custom callback)

from lifelines.utils import concordance_index
from keras.callbacks import Callback

class CIndex(Callback):
    """
    Callback that computes the C-index metric both on training and test data after each epoch.
    
    Arguments:
        train_x: Numpy array containing the training dataset.
        train_time: Numpy array with survival time for training dataset.
        train_event: Numpy array with survival event for training dataset.
        val_x: Numpy array containing the test dataset.
        val_time: Numpy array with survival time for test dataset.
        val_event: Numpy array with survival event for test dataset.
        filepath: path to save the model.
        years: number of years to compute c-index
        patience: patience for early-stop training
    """
    
    def __init__(self, train_x, train_time, train_event, val_x, val_time, val_event, filepath, years=2, patience=5):
        super(Callback, self).__init__()
        self.X_tr = train_x
        self.time_tr = train_time
        self.event_tr = train_event
        self.X_val = val_x
        self.time_val = val_time
        self.event_val = val_event
        self.years = years
        self.patience = patience
        self.best = 0.0
        self.wait = 0  #counter for patience
        self.best_rounds = 1
        self.counter = 0
        self.filepath = filepath
        
    def on_train_begin(self, logs={}):
        self.c_index_dict = {'c-index': [], 'val_c-index': []}

    def on_epoch_end(self, epoch, logs={}):
        self.counter +=1

        y_tr=self.model.predict_proba(self.X_tr,verbose=0)
        yr_surv=np.cumprod(y_tr[:,0:np.nonzero(breaks>365*self.years)[0][0]], axis=1)[:,-1]
        c_index_tr = concordance_index(self.time_tr, yr_surv, self.event_tr)
        y_val=self.model.predict_proba(self.X_val,verbose=0)
        yr_surv_val=np.cumprod(y_val[:,0:np.nonzero(breaks>365*self.years)[0][0]], axis=1)[:,-1]
        c_index_val = concordance_index(self.time_val, yr_surv_val, self.event_val)

        print('\tc-index: %s - val_c-index: %s' % (str(round(c_index_tr, 4)), str(round(c_index_val, 4))))
        
        if (c_index_val > self.best):
            self.c_index_dict['c-index'].append(c_index_tr)
            self.c_index_dict['val_c-index'].append(c_index_val)

            self.best = c_index_val
            self.best_rounds = self.counter
            self.wait = 0
            self.model.save_weights(self.filepath, overwrite=True)
        else:
            if self.wait >= self.patience: #no more patience, retrieve best model
                self.model.stop_training = True
                print('Best number of rounds: %d \nValidation C-Index: %f \n' % (self.best_rounds, self.best))
                self.model.load_weights(self.filepath)
            self.wait += 1

## NON-BRCA

### Bayesian Optimization (non-brca)

In [ ]:
from hyperopt import hp, fmin, tpe, hp, STATUS_OK, Trials

In [ ]:
space = {
    'conv_choice': hp.choice('conv_num_layers', 
                            [{'layers': 'two',
                             'units_2conv2': hp.choice('units_2conv2', [16, 32, 64, 128]),
                              '2kernel2': hp.choice('2kernel2', [3, 5, 10, 20]),
                              '2pool2': hp.choice('2pool2', [2, 3, 5]),
                             'dropout_2conv2': hp.choice('dropout_2conv2', [0.25, 0.5, 0.75])
                            },
                            {'layers': 'three',
                             'units_3conv2': hp.choice('units_3conv2', [16, 32, 64, 128]),
                             '3kernel2': hp.choice('3kernel2', [3, 5, 10, 20]),
                             '3pool2': hp.choice('3pool2', [2, 3, 5]),
                             'dropout_3conv2': hp.choice('dropout_3conv2', [0.25, 0.5, 0.75]),
                             'units_3conv3': hp.choice('units_3conv3', [16, 32, 64, 128]),
                             '3kernel3': hp.choice('kernel3', [3, 5, 10, 20]),
                             '3pool3': hp.choice('3pool3', [2, 3, 5]),
                             'dropout_3conv3': hp.choice('dropout_3conv3', [0.25, 0.5, 0.75])
                            }]
                            ),
    'dense_choice': hp.choice('dense_num_layers', 
                            [{'layers': 'two',
                             'units_2dense2': hp.choice('units_2dense2', [16, 32, 64, 128]),
                             'dropout_2dense2': hp.choice('dropout_2dense2', [0.25, 0.5, 0.75])
                            },
                            {'layers': 'three',
                             'units_3dense2': hp.choice('units_3dense2', [16, 32, 64, 128]),
                             'dropout_3dense2': hp.choice('dropout_3dense2', [0.25, 0.5, 0.75]),
                             'units_3dense3': hp.choice('units_3dense3', [16, 32, 64, 128]),
                             'dropout_3dense3': hp.choice('dropout_3dense3', [0.25, 0.5, 0.75])
                            }]
                            ),
    'units_conv1': hp.choice('units_conv1', [16, 32, 64, 128]),
    'kernel1': hp.choice('kernel1', [3, 5, 10, 20]),
    'pool1': hp.choice('pool1', [2, 3, 5]),
    'dropout_conv1': hp.choice('dropout_conv1', [0.25, 0.5, 0.75]),
    'units_dense1': hp.choice('units_dense1', [16, 32, 64, 128]),
    'dropout_dense1': hp.choice('dropout_dense1', [0.25, 0.5, 0.75]),
    'batch_size' : hp.choice('batch_size', [16, 32, 64, 128]),
    'nb_epochs' : 100,
    'lr': hp.loguniform('lr', np.log(0.0001), np.log(0.01)),
    'activation': 'relu'                                            
}

In [ ]:
def train_non_brca(params):
    print('Params testing: ', params)
    print('\n')
    
    # Prevent GPU memory allocation problems
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.per_process_gpu_memory_fraction = 0.4
    K.set_session(tf.Session(config=config))

    ## CREATE MODEL    
    model = Sequential()

    # Conv layer 1 ---
    model.add(Conv1D(filters=params['units_conv1'], 
                     kernel_size=params['kernel1'],
                     input_shape = (X_non_brca_train.shape[1],
                                    X_non_brca_train.shape[2])))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(MaxPool1D(pool_size=params['pool1']))
    model.add(Dropout(params['dropout_conv1']))

    if params['conv_choice']['layers'] == 'two':
        # Conv layer 2 ---
        model.add(Conv1D(filters=params['conv_choice']['units_2conv2'], 
                     kernel_size=params['conv_choice']['2kernel2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(MaxPool1D(pool_size=params['conv_choice']['2pool2']))
        model.add(Dropout(params['conv_choice']['dropout_2conv2']))
    elif params['conv_choice']['layers'] == 'three':
        # Conv layer 2 ---
        model.add(Conv1D(filters=params['conv_choice']['units_3conv2'], 
                     kernel_size=params['conv_choice']['3kernel2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(MaxPool1D(pool_size=params['conv_choice']['3pool2']))
        model.add(Dropout(params['conv_choice']['dropout_3conv2']))
        # Conv layer 3 ---
        model.add(Conv1D(filters=params['conv_choice']['units_3conv3'], 
                     kernel_size=params['conv_choice']['3kernel3']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(MaxPool1D(pool_size=params['conv_choice']['3pool3']))
        model.add(Dropout(params['conv_choice']['dropout_3conv3']))

    # Flatten layer ---
    model.add(Flatten())
    
    # Dense layer 1 ---
    model.add(Dense(params['units_dense1']))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dropout_dense1']))

    if params['dense_choice']['layers']== 'two':
        # Dense layer 2 ---
        model.add(Dense(params['dense_choice']['units_2dense2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dense_choice']['dropout_2dense2']))
    elif params['dense_choice']['layers']== 'three':
        # Dense layer 2 ---
        model.add(Dense(params['dense_choice']['units_3dense2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dense_choice']['dropout_3dense2']))
        # Dense layer 3 ---
        model.add(Dense(params['dense_choice']['units_3dense3']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dense_choice']['dropout_3dense3']))
        
    # Output layer ---
    model.add(Dense(n_intervals, activation='sigmoid'))

    optimizer = Adam(lr=params['lr'])

    model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizer)

    hist_c_index = CIndex(X_non_brca_train,
                        non_brca_clinical.iloc[train_index]['OS.time'],
                        non_brca_clinical.iloc[train_index]['OS'],
                        X_non_brca_val,
                        non_brca_clinical.iloc[val_index]['OS.time'],
                        non_brca_clinical.iloc[val_index]['OS'],
                        filepath = 'keras-models/common_best_non-brca.h5')

    callbacks = [hist_c_index]   

    batch = params['batch_size']
    n_epoch = params['nb_epochs']

    start = timer()
    model.fit(X_non_brca_train, 
              y_non_brca_train, 
              batch_size=batch, 
              epochs=n_epoch, 
              validation_data = [X_non_brca_val, y_non_brca_val],
              callbacks=callbacks,
              verbose=0)
    run_time = timer() - start

    predictions = model.predict_proba(X_non_brca_test,verbose=0)
    twoyr_surv=np.cumprod(predictions[:,0:np.nonzero(breaks>365*2)[0][0]], axis=1)[:,-1]
    score = concordance_index(non_brca_clinical.iloc[test_index]['OS.time'], twoyr_surv, non_brca_clinical.iloc[test_index]['OS'])
    
    return {'loss': score*(-1),
            'params': params,
            'train_time': run_time,
            'status': STATUS_OK, 
            'rounds':hist_c_index.best_rounds}

In [ ]:
from timeit import default_timer as timer
from multiprocessing import Pool
from hyperopt import STATUS_OK

def objective_non_brca(params):
    from sklearn.metrics import roc_auc_score
    
    from keras.models import Sequential
    from keras.layers import Input, Dense, Conv1D, MaxPool1D, Activation, Dropout, Flatten
    from keras import regularizers, optimizers
    from keras.layers.normalization import BatchNormalization
    from keras.callbacks import EarlyStopping

    from keras.regularizers import l1
    from keras.optimizers import Adam
   
    with Pool(1) as p:
            trial = p.apply(train_non_brca, args=(params, ))

    return trial

In [ ]:
%%time

trials_non_brca = Trials()

best_non_brca = fmin(objective_non_brca, space, algo=tpe.suggest, trials=trials_non_brca, max_evals=50)

print (best_non_brca)
print (trials_non_brca.best_trial)

import pickle

with open("keras-models/common_hyper_opt_non-brca.pkl", 'wb') as f:
    pickle.dump([trials_non_brca, best_non_brca], f)

In [1]:
import pickle

with open("keras-models/common_hyper_opt_non-brca.pkl", 'rb') as f:
    [trials_non_brca, best_non_brca] = pickle.load(f)

In [12]:
import numpy as np
np.std(trials_non_brca.losses())
np.mean(trials_non_brca.losses())

-0.6435101458161087

## BRCA

### Bayesian Optimization (brca)

In [ ]:
from hyperopt import hp, fmin, tpe, hp, STATUS_OK, Trials

In [ ]:
space = {
    'conv_choice': hp.choice('conv_num_layers', 
                            [{'layers': 'two',
                             'units_2conv2': hp.choice('units_2conv2', [16, 32, 64, 128]),
                              '2kernel2': hp.choice('2kernel2', [3, 5, 10, 20]),
                              '2pool2': hp.choice('2pool2', [2, 3, 5]),
                             'dropout_2conv2': hp.choice('dropout_2conv2', [0.25, 0.5, 0.75])
                            },
                            {'layers': 'three',
                             'units_3conv2': hp.choice('units_3conv2', [16, 32, 64, 128]),
                             '3kernel2': hp.choice('3kernel2', [3, 5, 10, 20]),
                             '3pool2': hp.choice('3pool2', [2, 3, 5]),
                             'dropout_3conv2': hp.choice('dropout_3conv2', [0.25, 0.5, 0.75]),
                             'units_3conv3': hp.choice('units_3conv3', [16, 32, 64, 128]),
                             '3kernel3': hp.choice('kernel3', [3, 5, 10, 20]),
                             '3pool3': hp.choice('3pool3', [2, 3, 5]),
                             'dropout_3conv3': hp.choice('dropout_3conv3', [0.25, 0.5, 0.75])
                            }]
                            ),
    'dense_choice': hp.choice('dense_num_layers', 
                            [{'layers': 'two',
                             'units_2dense2': hp.choice('units_2dense2', [16, 32, 64, 128]),
                             'dropout_2dense2': hp.choice('dropout_2dense2', [0.25, 0.5, 0.75])
                            },
                            {'layers': 'three',
                             'units_3dense2': hp.choice('units_3dense2', [16, 32, 64, 128]),
                             'dropout_3dense2': hp.choice('dropout_3dense2', [0.25, 0.5, 0.75]),
                             'units_3dense3': hp.choice('units_3dense3', [16, 32, 64, 128]),
                             'dropout_3dense3': hp.choice('dropout_3dense3', [0.25, 0.5, 0.75])
                            }]
                            ),
    'units_conv1': hp.choice('units_conv1', [16, 32, 64, 128]),
    'kernel1': hp.choice('kernel1', [3, 5, 10, 20]),
    'pool1': hp.choice('pool1', [2, 3, 5]),
    'dropout_conv1': hp.choice('dropout_conv1', [0.25, 0.5, 0.75]),
    'units_dense1': hp.choice('units_dense1', [16, 32, 64, 128]),
    'dropout_dense1': hp.choice('dropout_dense1', [0.25, 0.5, 0.75]),
    'batch_size' : hp.choice('batch_size', [16, 32, 64, 128]),
    'nb_epochs' : 100,
    'lr': hp.loguniform('lr', np.log(0.0001), np.log(0.01)),
    'activation': 'relu'                                            
}

In [ ]:
def train_brca(params):
    print('Params testing: ', params)
    print('\n')
    
    # Prevent GPU memory allocation problems
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.per_process_gpu_memory_fraction = 0.4
    K.set_session(tf.Session(config=config))

    ## CREATE MODEL    
    model = Sequential()

    # Conv layer 1 ---
    model.add(Conv1D(filters=params['units_conv1'], 
                     kernel_size=params['kernel1'],
                     input_shape = (X_brca_train.shape[1],
                                    X_brca_train.shape[2])))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(MaxPool1D(pool_size=params['pool1']))
    model.add(Dropout(params['dropout_conv1']))

    if params['conv_choice']['layers'] == 'two':
        # Conv layer 2 ---
        model.add(Conv1D(filters=params['conv_choice']['units_2conv2'], 
                     kernel_size=params['conv_choice']['2kernel2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(MaxPool1D(pool_size=params['conv_choice']['2pool2']))
        model.add(Dropout(params['conv_choice']['dropout_2conv2']))
    elif params['conv_choice']['layers'] == 'three':
        # Conv layer 2 ---
        model.add(Conv1D(filters=params['conv_choice']['units_3conv2'], 
                     kernel_size=params['conv_choice']['3kernel2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(MaxPool1D(pool_size=params['conv_choice']['3pool2']))
        model.add(Dropout(params['conv_choice']['dropout_3conv2']))
        # Conv layer 3 ---
        model.add(Conv1D(filters=params['conv_choice']['units_3conv3'], 
                     kernel_size=params['conv_choice']['3kernel3']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(MaxPool1D(pool_size=params['conv_choice']['3pool3']))
        model.add(Dropout(params['conv_choice']['dropout_3conv3']))

    # Flatten layer ---
    model.add(Flatten())
    
    # Dense layer 1 ---
    model.add(Dense(params['units_dense1']))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dropout_dense1']))

    if params['dense_choice']['layers']== 'two':
        # Dense layer 2 ---
        model.add(Dense(params['dense_choice']['units_2dense2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dense_choice']['dropout_2dense2']))
    elif params['dense_choice']['layers']== 'three':
        # Dense layer 2 ---
        model.add(Dense(params['dense_choice']['units_3dense2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dense_choice']['dropout_3dense2']))
        # Dense layer 3 ---
        model.add(Dense(params['dense_choice']['units_3dense3']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dense_choice']['dropout_3dense3']))
        
    # Output layer ---
    model.add(Dense(n_intervals, activation='sigmoid'))

    optimizer = Adam(lr=params['lr'])

    model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizer)

    hist_c_index = CIndex(X_brca_train,
                        brca_clinical.iloc[train_index]['OS.time'],
                        brca_clinical.iloc[train_index]['OS'],
                        X_brca_val,
                        brca_clinical.iloc[val_index]['OS.time'],
                        brca_clinical.iloc[val_index]['OS'],
                        filepath = 'keras-models/common_best_brca.h5')

    callbacks = [hist_c_index]   

    batch = params['batch_size']
    n_epoch = params['nb_epochs']

    start = timer()
    model.fit(X_brca_train, 
              y_brca_train, 
              batch_size=batch, 
              epochs=n_epoch, 
              validation_data = [X_brca_val, y_brca_val],
              callbacks=callbacks,
              verbose=0)
    run_time = timer() - start

    predictions = model.predict_proba(X_brca_test,verbose=0)
    twoyr_surv=np.cumprod(predictions[:,0:np.nonzero(breaks>365*2)[0][0]], axis=1)[:,-1]
    score = concordance_index(brca_clinical.iloc[test_index]['OS.time'], twoyr_surv, brca_clinical.iloc[test_index]['OS'])
    
    return {'loss': score*(-1),
            'params': params,
            'train_time': run_time,
            'status': STATUS_OK, 
            'rounds':hist_c_index.best_rounds}

In [ ]:
from timeit import default_timer as timer
from multiprocessing import Pool
from hyperopt import STATUS_OK

def objective_brca(params):
    from sklearn.metrics import roc_auc_score
    
    from keras.models import Sequential
    from keras.layers import Input, Dense, Conv1D, MaxPool1D, Activation, Dropout, Flatten
    from keras import regularizers, optimizers
    from keras.layers.normalization import BatchNormalization
    from keras.callbacks import EarlyStopping

    from keras.regularizers import l1
    from keras.optimizers import Adam
   
    with Pool(1) as p:
            trial = p.apply(train_brca, args=(params, ))

    return trial

In [ ]:
%%time

trials_brca = Trials()

best_brca = fmin(objective_brca, space, algo=tpe.suggest, trials=trials_brca, max_evals=50)

print (best_brca)
print (trials_brca.best_trial)

import pickle

with open("keras-models/common_hyper_opt_brca.pkl", 'wb') as f:
    pickle.dump([trials_brca, best_brca], f)

In [ ]:
import pickle

with open("keras-models/common_hyper_opt_brca.pkl", 'rb') as f:
    [trials_brca, best_brca] = pickle.load(f)

In [ ]:
trials_brca.best_trial

## BRCA (fine tuning) 

#### Train a non-brca model with all non-brca data

In [45]:
import pickle

with open("keras-models/common_hyper_opt_non-brca.pkl", 'rb') as f:
    [trials_non_brca, best_non_brca] = pickle.load(f)

params = trials_non_brca.best_trial['result']['params']

In [48]:
from timeit import default_timer as timer

# Prevent GPU memory allocation problems
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.4
K.set_session(tf.Session(config=config))

## CREATE MODEL    
model = Sequential()

# Conv layer 1 ---
model.add(Conv1D(filters=params['units_conv1'], 
                 kernel_size=params['kernel1'],
                 input_shape = (X_non_brca.shape[1],
                                X_non_brca.shape[2])))
model.add(BatchNormalization())
model.add(Activation(params['activation']))
model.add(MaxPool1D(pool_size=params['pool1']))
model.add(Dropout(params['dropout_conv1']))

if params['conv_choice']['layers'] == 'two':
    # Conv layer 2 ---
    model.add(Conv1D(filters=params['conv_choice']['units_2conv2'], 
                 kernel_size=params['conv_choice']['2kernel2']))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(MaxPool1D(pool_size=params['conv_choice']['2pool2']))
    model.add(Dropout(params['conv_choice']['dropout_2conv2']))
elif params['conv_choice']['layers'] == 'three':
    # Conv layer 2 ---
    model.add(Conv1D(filters=params['conv_choice']['units_3conv2'], 
                 kernel_size=params['conv_choice']['3kernel2']))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(MaxPool1D(pool_size=params['conv_choice']['3pool2']))
    model.add(Dropout(params['conv_choice']['dropout_3conv2']))
    # Conv layer 3 ---
    model.add(Conv1D(filters=params['conv_choice']['units_3conv3'], 
                 kernel_size=params['conv_choice']['3kernel3']))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(MaxPool1D(pool_size=params['conv_choice']['3pool3']))
    model.add(Dropout(params['conv_choice']['dropout_3conv3']))

# Flatten layer ---
model.add(Flatten())

# Dense layer 1 ---
model.add(Dense(params['units_dense1']))
model.add(BatchNormalization())
model.add(Activation(params['activation']))
model.add(Dropout(params['dropout_dense1']))

if params['dense_choice']['layers']== 'two':
    # Dense layer 2 ---
    model.add(Dense(params['dense_choice']['units_2dense2']))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dense_choice']['dropout_2dense2']))
elif params['dense_choice']['layers']== 'three':
    # Dense layer 2 ---
    model.add(Dense(params['dense_choice']['units_3dense2']))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dense_choice']['dropout_3dense2']))
    # Dense layer 3 ---
    model.add(Dense(params['dense_choice']['units_3dense3']))
    model.add(BatchNormalization())
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dense_choice']['dropout_3dense3']))

# Output layer ---
model.add(Dense(n_intervals, activation='sigmoid'))

optimizer = Adam(lr=params['lr'])

model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizer)

batch = params['batch_size']
n_epoch = params['nb_epochs']

start = timer()
model.fit(X_non_brca, 
          y_non_brca, 
          batch_size=batch, 
          epochs=n_epoch)
run_time = timer() - start


Epoch 1/100
9224/9224 [==============================] - 8s 889us/step - loss: 1.9796
Epoch 2/100
9224/9224 [==============================] - 7s 705us/step - loss: 1.6039
Epoch 3/100
9224/9224 [==============================] - 7s 721us/step - loss: 1.5849
Epoch 4/100
9224/9224 [==============================] - 7s 712us/step - loss: 1.5634
Epoch 5/100
9224/9224 [==============================] - 7s 718us/step - loss: 1.5560
Epoch 6/100
9224/9224 [==============================] - 6s 704us/step - loss: 1.5343
Epoch 7/100
9224/9224 [==============================] - 7s 717us/step - loss: 1.5182
Epoch 8/100
9224/9224 [==============================] - 7s 706us/step - loss: 1.5194
Epoch 9/100
9224/9224 [==============================] - 7s 721us/step - loss: 1.5142
Epoch 10/100
9224/9224 [==============================] - 7s 706us/step - loss: 1.5010
Epoch 11/100
9224/9224 [==============================] - 7s 717us/step - loss: 1.5003
Epoch 12/100
9224/9224 [============================

In [49]:
predictions = model.predict_proba(X_non_brca,verbose=0)
twoyr_surv=np.cumprod(predictions[:,0:np.nonzero(breaks>365*2)[0][0]], axis=1)[:,-1]
score = concordance_index(non_brca_clinical['OS.time'], twoyr_surv, non_brca_clinical['OS'])

print(score)

0.78003794309897


In [50]:
model.save('keras-models/model_non-brca_all_data.h5')

#### Bayesian optimization (BRCA finetuning)

In [45]:
from hyperopt import hp, fmin, tpe, hp, STATUS_OK, Trials

In [46]:
space_fine_tuning = {
    'choice': hp.choice('num_layers', 
                            [{'layers': 'one'},
                            {'layers': 'two',
                             'units_2dense1': hp.choice('units_2dense1', [16, 32, 64, 128, 256]),
                             'dropout_2dense1': hp.choice('dropout_2dense1', [0.25, 0.5, 0.75])
                            },
                            {'layers': 'three',
                             'units_3dense1': hp.choice('units_3dense1', [16, 32, 64, 128, 256]),
                             'dropout_3dense1': hp.choice('dropout_3dense1', [0.25, 0.5, 0.75]),
                             'units_3dense2': hp.choice('units_3dense2', [16, 32, 64, 128, 256]),
                             'dropout_3dense2': hp.choice('dropout_3dense2', [0.25, 0.5, 0.75])
                            }]
                            ),
    'units_output': n_intervals,
    'batch_size' : hp.choice('batch_size', [16, 32, 64, 128]),
    'nb_epochs' : 100,
    'lr': hp.loguniform('lr', np.log(0.0001), np.log(0.01)),
    'activation_hidden': 'relu',
    'activation_output': 'sigmoid'
}

In [47]:
import nnet_survival

In [48]:
def train_fine_tuning(params):
    print('Params testing: ', params)
    print('\n')

    # Prevent GPU memory allocation problems
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.per_process_gpu_memory_fraction = 0.4
    K.set_session(tf.Session(config=config))

    from keras.models import load_model

    non_brca_model = load_model('keras-models/model_non-brca_all_data.h5', custom_objects={'loss': nnet_survival.surv_likelihood(n_intervals)})

    import pickle
    with open("keras-models/common_hyper_opt_non-brca.pkl", 'rb') as f:
        [trials_non_brca, best_non_brca] = pickle.load(f)
    
    if trials_non_brca.best_trial['result']['params']['dense_choice']['layers'] == 'two':
        pop_layers = 6
    elif trials_non_brca.best_trial['result']['params']['dense_choice']['layers'] == 'three':
        pop_layers = 9
    
    for i in range(pop_layers):
        non_brca_model.pop()    
    
    for layer in non_brca_model.layers:
         layer.trainable = False
    
    model = Sequential()
    
    model.add(non_brca_model)

    if params['choice']['layers']== 'two':
        # Dense layer 1 ---
        model.add(Dense(params['choice']['units_2dense1']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation_hidden']))
        model.add(Dropout(params['choice']['dropout_2dense1']))
    elif params['choice']['layers']== 'three':
        # Dense layer 1 ---
        model.add(Dense(params['choice']['units_3dense1']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation_hidden']))
        model.add(Dropout(params['choice']['dropout_3dense1']))
        # Dense layer 2 ---
        model.add(Dense(params['choice']['units_3dense2']))
        model.add(BatchNormalization())
        model.add(Activation(params['activation_hidden']))
        model.add(Dropout(params['choice']['dropout_3dense2']))
        
    # Output layer ---
    model.add(Dense(params['units_output'], activation=params['activation_output']))

    optimizer = Adam(lr=params['lr'])
    
    model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizer)
    
    #model.summary()
    
    batch = params['batch_size']
    n_epoch = params['nb_epochs']

    hist_c_index = CIndex(X_brca_train,
                    brca_clinical.iloc[train_index]['OS.time'],
                    brca_clinical.iloc[train_index]['OS'],
                    X_brca_val,
                    brca_clinical.iloc[val_index]['OS.time'],
                    brca_clinical.iloc[val_index]['OS'],
                    filepath = 'keras-models/common_best_fine-tuning_sigmoid.h5')

    callbacks = [hist_c_index]    
    
    ## Fit model using training data
    start = timer()
    model.fit(X_brca_train, 
              y_brca_train, 
              batch_size=batch, 
              epochs=n_epoch, 
              validation_data = [X_brca_val, y_brca_val],
              callbacks=callbacks,
              verbose=0)
    run_time = timer() - start
    
    predictions = model.predict_proba(X_brca_test,verbose=0)
    twoyr_surv=np.cumprod(predictions[:,0:np.nonzero(breaks>365*2)[0][0]], axis=1)[:,-1]
    score = concordance_index(brca_clinical.iloc[test_index]['OS.time'], twoyr_surv, brca_clinical.iloc[test_index]['OS'])

    # Return loss
    return {'loss': score*(-1),
            'params': params,
            'train_time': run_time,
            'status': STATUS_OK, 
            'rounds':hist_c_index.best_rounds}

In [49]:
from timeit import default_timer as timer
from multiprocessing import Pool
from hyperopt import STATUS_OK

def objective_fine_tuning(params):
    from sklearn.metrics import roc_auc_score
    
    from keras.models import Sequential
    from keras.layers import Input, Dense, Conv2D, MaxPool2D, Activation, Dropout, Flatten
    from keras import regularizers, optimizers
    from keras.layers.normalization import BatchNormalization
    from keras.callbacks import EarlyStopping

    from keras.regularizers import l1
    from keras.optimizers import Adam
       
    with Pool(1) as p:
            trial = p.apply(train_fine_tuning, args=(params, ))

    return trial

In [50]:
trials_fine_tuning = Trials()

best_fine_tuning = fmin(fn=objective_fine_tuning, space=space_fine_tuning, algo=tpe.suggest, trials=trials_fine_tuning, max_evals=100)

print (best_fine_tuning)
print (trials_fine_tuning.best_trial)

Params testing:                                      
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 16, 'choice': {'dropout_3dense1': 0.25, 'dropout_3dense2': 0.5, 'layers': 'three', 'units_3dense1': 32, 'units_3dense2': 64}, 'lr': 0.0010927272376659465, 'nb_epochs': 100, 'units_output': 39}
  0%|          | 0/100 [00:00<?, ?it/s, best loss: ?]WARNING:tensorflow:From /home/mcabello/almacen/miniconda3/envs/maria_env/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:497: calling conv1d (from tensorflow.python.ops.nn_ops) with data_format=NHWC is deprecated and will be removed in a future version.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
	c-index: 0.3885 - val_c-index: 0.4311               
	c-index: 0.4091 - val_c-index: 0.3775               
	c-index: 0.4278 - val_c-index: 0.3699               
	c-index: 0.4424 - val_c-index: 0.3675               
	c-index: 0.4452 - val_c-index: 0.3579               

	c-index: 0.4899 - val_c-index: 0.6967                                         
	c-index: 0.4894 - val_c-index: 0.6967                                         
	c-index: 0.4894 - val_c-index: 0.6976                                         
	c-index: 0.4896 - val_c-index: 0.6986                                         
	c-index: 0.4896 - val_c-index: 0.6981                                         
	c-index: 0.4893 - val_c-index: 0.6967                                         
	c-index: 0.4891 - val_c-index: 0.6957                                         
	c-index: 0.4888 - val_c-index: 0.6971                                         
	c-index: 0.4888 - val_c-index: 0.6971                                         
	c-index: 0.4891 - val_c-index: 0.6976                                         
Best number of rounds: 17                                                      
Validation C-Index: 0.698565 

Params testing:                                                                
{'activat

	c-index: 0.5162 - val_c-index: 0.4316                                          
	c-index: 0.4905 - val_c-index: 0.5201                                          
	c-index: 0.4866 - val_c-index: 0.5981                                          
	c-index: 0.486 - val_c-index: 0.6431                                           
	c-index: 0.4877 - val_c-index: 0.6675                                          
	c-index: 0.4895 - val_c-index: 0.6818                                          
	c-index: 0.4899 - val_c-index: 0.6847                                          
	c-index: 0.4894 - val_c-index: 0.6866                                          
	c-index: 0.4896 - val_c-index: 0.69                                            
	c-index: 0.4892 - val_c-index: 0.6914                                          
	c-index: 0.4896 - val_c-index: 0.6928                                          
	c-index: 0.4899 - val_c-index: 0.6952                                          
	c-index: 0.4898 - val_c-ind

	c-index: 0.5343 - val_c-index: 0.6392                                          
	c-index: 0.5344 - val_c-index: 0.6407                                          
	c-index: 0.5339 - val_c-index: 0.6426                                          
	c-index: 0.5333 - val_c-index: 0.6435                                          
	c-index: 0.5333 - val_c-index: 0.6455                                          
	c-index: 0.5346 - val_c-index: 0.6478                                          
	c-index: 0.5354 - val_c-index: 0.6478                                          
	c-index: 0.536 - val_c-index: 0.6474                                           
	c-index: 0.5352 - val_c-index: 0.6431                                          
	c-index: 0.5348 - val_c-index: 0.644                                           
	c-index: 0.535 - val_c-index: 0.6455                                           
	c-index: 0.5349 - val_c-index: 0.6445                                          
Best number of rounds: 28   

	c-index: 0.4512 - val_c-index: 0.5311                                          
	c-index: 0.4425 - val_c-index: 0.4813                                          
	c-index: 0.4574 - val_c-index: 0.4373                                          
	c-index: 0.4708 - val_c-index: 0.4225                                          
	c-index: 0.4791 - val_c-index: 0.3971                                          
	c-index: 0.4899 - val_c-index: 0.3986                                          
Best number of rounds: 2                                                        
Validation C-Index: 0.564115 

Params testing:                                                                 
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 16, 'choice': {'dropout_3dense1': 0.75, 'dropout_3dense2': 0.75, 'layers': 'three', 'units_3dense1': 256, 'units_3dense2': 16}, 'lr': 0.00022463985603026024, 'nb_epochs': 100, 'units_output': 39}
 26%|██▌       | 26/100 [06:10<14:55, 12.11s/it, 

...

	c-index: 0.5396 - val_c-index: 0.5096                                          
	c-index: 0.5373 - val_c-index: 0.5129                                          
	c-index: 0.5361 - val_c-index: 0.5134                                          
	c-index: 0.5347 - val_c-index: 0.5172                                          
	c-index: 0.5331 - val_c-index: 0.5244                                          
	c-index: 0.5312 - val_c-index: 0.5225                                          
	c-index: 0.5288 - val_c-index: 0.522                                           
	c-index: 0.5264 - val_c-index: 0.5211                                          
	c-index: 0.5246 - val_c-index: 0.523                                           
	c-index: 0.5215 - val_c-index: 0.5282                                          
	c-index: 0.5201 - val_c-index: 0.5268                                          
	c-index: 0.5181 - val_c-index: 0.5254                                          
	c-index: 0.5154 - val_c-ind

 72%|███████▏  | 72/100 [15:02<04:31,  9.70s/it, best loss: -0.6743421052631579]WARNING:tensorflow:From /home/mcabello/almacen/miniconda3/envs/maria_env/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:497: calling conv1d (from tensorflow.python.ops.nn_ops) with data_format=NHWC is deprecated and will be removed in a future version.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
	c-index: 0.4958 - val_c-index: 0.5957                                          
	c-index: 0.4905 - val_c-index: 0.5919                                          
	c-index: 0.497 - val_c-index: 0.5694                                           
	c-index: 0.5031 - val_c-index: 0.5373                                          
	c-index: 0.5153 - val_c-index: 0.4947                                          
	c-index: 0.5278 - val_c-index: 0.4574                                          
	c-index: 0.5265 - val_c-index: 0.4091                                         

	c-index: 0.4893 - val_c-index: 0.6986                                          
	c-index: 0.489 - val_c-index: 0.7                                              
	c-index: 0.489 - val_c-index: 0.6995                                           
	c-index: 0.4893 - val_c-index: 0.699                                           
	c-index: 0.4894 - val_c-index: 0.699                                           
	c-index: 0.489 - val_c-index: 0.6986                                           
	c-index: 0.4888 - val_c-index: 0.699                                           
	c-index: 0.4885 - val_c-index: 0.6995                                          
Best number of rounds: 13                                                       
Validation C-Index: 0.700000 

Params testing:                                                                 
{'activation_hidden': 'relu', 'activation_output': 'sigmoid', 'batch_size': 16, 'choice': {'dropout_3dense1': 0.25, 'dropout_3dense2': 0.5, 'layers': 'three', 

`NHWC` for data_format is deprecated, use `NWC` instead
	c-index: 0.5173 - val_c-index: 0.412                                           
	c-index: 0.5106 - val_c-index: 0.3694                                          
	c-index: 0.4824 - val_c-index: 0.3861                                          
	c-index: 0.5007 - val_c-index: 0.4282                                          
	c-index: 0.5222 - val_c-index: 0.3995                                          
	c-index: 0.5556 - val_c-index: 0.5129                                          
	c-index: 0.5603 - val_c-index: 0.5804                                          
	c-index: 0.5626 - val_c-index: 0.5938                                          
	c-index: 0.5405 - val_c-index: 0.6234                                          
	c-index: 0.5642 - val_c-index: 0.6455                                          
	c-index: 0.5949 - val_c-index: 0.5952                                          
	c-index: 0.5798 - val_c-index: 0.6134               

 92%|█████████▏| 92/100 [19:10<01:52, 14.07s/it, best loss: -0.7212918660287081]WARNING:tensorflow:From /home/mcabello/almacen/miniconda3/envs/maria_env/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:497: calling conv1d (from tensorflow.python.ops.nn_ops) with data_format=NHWC is deprecated and will be removed in a future version.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
	c-index: 0.4909 - val_c-index: 0.6914                                          
	c-index: 0.4914 - val_c-index: 0.6947                                          
	c-index: 0.4906 - val_c-index: 0.6962                                          
	c-index: 0.49 - val_c-index: 0.6981                                            
	c-index: 0.4892 - val_c-index: 0.6995                                          
	c-index: 0.4886 - val_c-index: 0.701                                           
	c-index: 0.4884 - val_c-index: 0.7014                                         

In [51]:
import pickle

with open("keras-models/common_hyper_opt_fine-tuning_sigmoid.pkl", 'wb') as f:
    pickle.dump([trials_fine_tuning, best_fine_tuning], f)